In [87]:
import sys
print (sys.version)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.basemap import Basemap
from itertools import chain
import math
%matplotlib inline

3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]



# Project WP1-2
### code 3: Update the port file "portsInfo_new.csv" in terms of the feature "Port Group".
<span style='font-family:"Times New Roman"'> <span styel=''> 

There are some defects when searching and assigning a main port for terminals within some distance in the former file "portsInfo_new.csv". This code is to udpdate the feature "Port Group", replacing the old column in the file. By using the resursive strategy,an efficient and accurate searching is obtained. 
 
The original dataset is "portsInfo_new.csv";
### Basic steps: 
    a) Load data;
    b) Update main port (recursive)
    c) double check data
    d) rename and save data

### a) Load data

In [88]:
df_ports=pd.read_csv("C:/posDoc-SDU-Denmark/SDU_research/shippingLabwp1/cleanedData/portsInfo_new.csv")
df_ports["Lat"]=df_ports["Lat"].apply(lambda x: float(x.replace(",","."))) # string to float
df_ports["Lon"]=df_ports["Lon"].apply(lambda x: float(x.replace(",",".")))
print ("the length of portdataset:",len(df_ports))
df_ports.tail(20)

the length of portdataset: 3288


,ix,Region,Country,Port Group,Terminal,Lat,Lon,num of call all,num of call end,num of call start
3268,2659,SE Asia,unknown,Bunga Raya Marine Terminal,South East Asia,7.12922,103.42850,4,3,1
3269,2611,Caribbean,unknown,Caribbean,Caribbean,15.35787,-76.39933,3,3,0
3270,2104,Gulf,unknown,Cayo Arcas,Gulf of Mexico,19.23552,-91.87366,12,9,3
3271,2215,NE Asia,unknown,East China Sea,East China Sea,34.09471,124.38974,7,7,0
3272,2744,Mediterranean,unknown,Falconara,East Mediterranean,43.63373,13.96120,2,2,0
3273,2901,Mediterranean,unknown,Fertile Crescent,Fertile Crescent,32.84876,31.84852,1,1,0
3274,2739,South Asia,unknown,India West Coast,India West Coast,4.79132,76.20889,2,2,0
3275,2975,East Africa,unknown,Mahe,Indian Ocean,-5.41585,55.04807,1,1,0
3276,2919,Oceania,unknown,Marsden Point Refinery,New Zealand,-35.73105,176.29282,1,1,0
3277,3183,Baltic,unknown,Naantali,Baltic,59.13422,21.12515,1,1,0


# b) Functions to define similar groups (recursive way), and assign a main port for each group.
Specifically, the terminal groups are first grouped by every region. Then, the searching for similar terminals is within the grouped region one by one based on a sorted sequence (sorted by num of call all). 

In [89]:

def dist(lat1,lat2,lon1,lon2):
    """
    compute the great-circle distance between two locations on earth.
    lat1,lon1: location information of location 1; in degrees
    lat2,lon2: location information of location 2; in degrees
    
    return: numeric; distance, in km
    """ 
    from math import radians, cos, sin, asin,sqrt
    R=6371  # km
    
    #  turn to radias
    lon1=radians(lon1) 
    lon2=radians(lon2)
    lat1=radians(lat1)
    lat2=radians(lat2)
    
    # Haversine formula
    dlon=lon2-lon1
    dlat=lat2-lat1
    a=sin(dlat/2)**2+cos(lat1)*cos(lat2)*sin(dlon/2)**2
    
    c = 2 * asin(sqrt(a))
    d=R*c # km
    return d

def similar_terminal(df, ix1, lat1, lon1,dist_ref=100 * 1.852):
    """
    Given a location (ix1,lat1, lon1), compute the distance between this single location and all other locations in dataframe.
    Then, extract all the ids of similar terminals when dist is not larger than dist_ref.
    
    df:dataFrame
    ix1,lat1,lon1: index of each row, latitude, longitude
    dist_ref: reference distance,the default is 185.2km
    return:ids dict; terminals dict
    """
    similars_ids_single = {}
    similars_terminals_single = {}

    # loop to find the suitable ids
    for ix2, lat2, lon2 in zip(df.index, df.Lat, df.Lon):
        dist_temp = dist(lat1, lat2, lon1, lon2)
        if dist_temp <= dist_ref:
            similars_ids_single.setdefault(ix1, []).append(ix2)
            similars_terminals_single.setdefault(df.loc[ix1, 'Terminal'], []).append(df.loc[ix2, 'Terminal'])

    if (similars_ids_single=={}) or (similars_terminals_single=={}):
        return "No",ix1,similars_ids_single, similars_terminals_single
    else:
        return "Yes",ix1,similars_ids_single, similars_terminals_single

        
def group_region(df,region):
    """
    Using recusive to find the similar terminals in a given region.
    df: dataframe including all port info
    region: string
    
    return: lists containg all similar terminals, and their ids. [{mainPort:[terminal1,terminal2...]},{...}]
    """
    df_region=df[df.Region==region].sort_values(axis=0,by=["num of call all"],ascending=False,inplace=False)
    terminals =df_region.Terminal.values
    
    groupsTerminal_all=[]
    groupsId_all=[]
    
    def group_help(df,region,terminals_region):
        # help function for recursive.
        # terminals: sorted list, by "num of calls all"
        # remember that the terminals should be sorted first to make sure that all popular terminals are used as the main port

        if terminals_region == []:
            return groupsTerminal_all

        mainTerminal=terminals_region[0] # the first element of terminals always be the most porpular terminal
        ix1=df.index[df.Terminal==mainTerminal].to_list()[0]
        lat1=float(df.loc[ix1,"Lat"])
        lon1=float(df.loc[ix1,"Lon"])

        df_region=df[df.Region==region]
        flag,ix1,similars_ids_single, similars_terminals_single=similar_terminal(df_region, 
                                                                                      ix1, lat1, lon1,dist_ref=100 * 1.852)
        rest_terminals = [terminal for terminal in terminals_region if terminal not in similars_terminals_single[mainTerminal]]
        groupsTerminal_all.append(similars_terminals_single)
        groupsId_all.append(similars_ids_single)

        return group_help(df_region,region,rest_terminals)

    group_help(df,region,terminals)
    
    return groupsTerminal_all,groupsId_all


def group_all(df):
    """
    compute the similar terminals in every region, and return as a list
    df: dataframe
    
    return: lists containg all similar terminals, and their ids. [{mainPort:[terminal1,terminal2...]},{...}]
    """
    
    terminals_all=[]
    terminalsID_all=[]
    
    regions=list(set(df.Region.to_list()))
    for region in regions:
        groupsTerminal_region,groupsId_region=group_region(df,region)
        terminals_all.extend(groupsTerminal_region)
        terminalsID_all.extend(groupsId_region)
    return terminals_all,terminalsID_all


def groupCommon(df,portCommon="Rio Cullen",terminals_similar=[1231,1556, 1687, 1897, 1934],extra_col="Main_port",sortValue="num of call all"):
    """
    Group dataframe with given idx, and sort it in a descend way.
    sorted the dataframe by sortValue in a descend way, Add extra column 
    
    df: dataframe
    portCommon: string, terminal used as the common port name of a group of close terminals
    terminals_similar: list; all ids of similar terminals with portCommon,including portCommon itself. 
    sortValue: string, used for sorting of dataframe
    
    return:dataframe of a group of similar terminals
    """
    df_similar=df.loc[terminals_similar]
    df_similar.sort_values(axis=0,by=[sortValue],ascending=False,inplace=True)
    df_similar[extra_col]=portCommon
    
    return df_similar

def add_mainPorts(df,mainDict,similars_ids_all):
    """
    concatenate all grouped ports
    
    df: dataframe
    mainDict: dict, with key : main port; value: list of similar terminals
    similars_ids_all: list, containing dicts of all(id:[similar terminals])
    
    return: a dataframe with new column "Main_port" 
    """
    df_new=pd.DataFrame({})
    similars_ids_all=listToDict(similars_ids_all)
    
    for terminal in mainDict.keys():
        terminal_key=df.index[df.Terminal==terminal].tolist()[0]
        ixx=similars_ids_all[terminal_key]
        df_similar_temp=groupCommon(df,portCommon=terminal,terminals_similar=ixx,
                                    extra_col="Main_ports",sortValue="num of call all")
        df_new=pd.concat([df_new,df_similar_temp],axis=0)
        
    return df_new

def listToDict(list1):
    # swith a list to a dict
    # to dicts
    dicts={}
    for ix,elem in enumerate(list1):
        dicts.update(elem)
    return dicts

def remove_dups(df):
    """
    Find and drop the duplicates (keep the first one)
    df: dataFrame with duplicates in "Terminal"
    return: dataFrame without duplicate "Terminal"
    """
    
    # find and drop the first duplicate
    df_noDup=df.drop_duplicates(subset=["Terminal"],keep=False,inplace=False)
    
    pd_dupTs=pd.concat(g for _, g in df.groupby("Terminal") if len(g) > 1)
    pd_dupTs.drop_duplicates(subset=["Terminal"],keep='first',inplace=True)
    
    return pd_dupTs,df_noDup

def csvToxlsx(given_fileNames=['ports_update1']):
    # save data from csv to excel format
    from pandas.io.excel import ExcelWriter
    import pandas

    csv_files = given_fileNames
    with ExcelWriter("C:/posDoc-SDU-Denmark/SDU_research/shippingLabwp1/savedFile/"+"ports_update1"+"."+"xlsx") as ew:
        for csv_file in csv_files:
            pandas.read_csv("C:/posDoc-SDU-Denmark/SDU_research/shippingLabwp1/savedFile/"+csv_file+"."+"csv").to_excel(ew, sheet_name=csv_file)

### a few test cases for funcitons.

In [90]:
# a test case for funcion group_region()
groupsTerminal_all,groupsId_all=group_region(df_ports,"English channel")
print ("The terminals of the region 'English channel' can be grouped:")
print ("---------------------------------------------------------------------------------------------------------------------")
print (groupsTerminal_all[:1])
print ("---------------------------------------------------------------------------------------------------------------------")
print (groupsId_all[:1])

The terminals of the region 'English channel' can be grouped:
---------------------------------------------------------------------------------------------------------------------
[{'Antwerp': ['Antwerp', 'Vopak Terminal Europoort', 'Rotterdam', 'SEA-Tank Terminal Antwerp', 'Oiltanking Amsterdam Terminal', 'Vopak Terminal Amsterdam Westpoort', 'Shell Europoort Terminal', 'Maasvlakte Oil Terminal', 'SEA Tank 510', 'Euro Tank Terminals', 'Vopak Terminal Botlek', 'Koole Tanskstorage Minerals', 'Amsterdam', 'Vopak Terminal TTR', 'Team Terminal', 'Maatschap Europoort Terminal', 'BP Refinery Rotterdam', 'ETA Terminal', 'Vlissingen', 'Oiltanking Stolthaven Antwerp', 'Inter Terminals Amsterdam', 'Zenith Energy Amsterdam Terminal', 'Koole Pernis Terminal', 'Shell Pernis Refinery', 'Gunvor Petroleum Antwerpen', 'Ghent', 'Esso Botlek Terminal', 'Vopak Terminal Vlaardingen', 'Noord Natie Terminal', 'ATPC', 'Vesta Terminal Antwerp', 'Total Antwerp Refinery', 'Oiltanking Terneuzen', 'Dunkirk', 'Rubi

<ipython-input-89-9ec79959a161>:72: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if terminals_region == []:


In [91]:
# group all terminals in each region
terminals_all,terminalsID_all=group_all(df_ports)

# add "Main_port" based on grouped terminals
terminals_all=listToDict(terminals_all)
df_new=add_mainPorts(df_ports,terminals_all,terminalsID_all)

#remove duplicates rows.
df_dups,df_new_nodup=remove_dups(df_new)
df_new=pd.concat([df_dups,df_new_nodup],axis=0)
df_new.sort_values(axis=0,by=["num of call all"],ascending=False,inplace=True)

<ipython-input-89-9ec79959a161>:72: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if terminals_region == []:


# c) Double check the dataset 

In [92]:
print ("---------------------------------------------------------------------------------------------------------------------")
print ("Overall length of data:",len(df_new))
df_new.head()

---------------------------------------------------------------------------------------------------------------------
Overall length of data: 3288


,ix,Region,Country,Port Group,Terminal,Lat,Lon,num of call all,num of call end,num of call start,Main_ports
0,0,SE Asia,Singapore,Singapore,Singapore,1.20470,103.67232,14768,7365,7403,Singapore
1,1,Baltic,Russia,PTP Terminal,PTP Terminal,60.32058,28.72898,7566,3781,3785,PTP Terminal
2,2,Black sea,Russia,Novorossiysk,Novorossiysk,44.72680,37.78154,7524,3762,3762,Novorossiysk
3,3,NE Asia,South Korea,Yeosu,Yeosu,34.71473,127.81125,7078,3547,3531,Yeosu
4,4,Gulf,United States of America,Houston,Houston,29.23550,-94.62350,6746,3375,3371,Houston


In [93]:
def check_missData(df):
    """
    check the missing data of dataFrame.
    return: a dataFrame describing missing data percentage
    sum(): count the number of missing data in each column
    isnull(): return a dataFrame with True and False   
    """
    miss_tot=df.isnull().sum().sort_values(ascending=False)
    counts_all=df.isnull().count() # count all the elements, including the missed elements
    miss_per=((df.isnull().sum())*100/counts_all).sort_values(ascending=False)
    miss_all=pd.concat([miss_tot,miss_per],axis=1,keys=['TotalNum','TotalPerc(\%)'])
    return miss_all

def check_valsCount(df):
    """
    check the number of counts of each value in dataframe
    df: a dataframe
    return: a list including all the counts of each column
    """
    cols=df.columns
    counts_all=[]
    for col in cols:
        counts=df.loc[:,col].value_counts()
        counts_all.append(counts)
        print (counts)
        print ('\n')
    return counts_all

check_missData(df_new)

,TotalNum,TotalPerc(\%)
Main_ports,0,0.0
num of call start,0,0.0
num of call end,0,0.0
num of call all,0,0.0
Lon,0,0.0
Lat,0,0.0
Terminal,0,0.0
Port Group,0,0.0
Country,0,0.0
Region,0,0.0


In [94]:
# Updated rows/terminals checking btw new port and old port
ix_dis=[]
term_dis=[]

for i, ele1,ele2 in zip(df_new.index,df_new["Port Group"],df_new["Main_ports"]):
    #print (i,ele1,ele2)
    if ele1!=ele2:
        term_temp=df_new.loc[i,"Terminal"]
        ix_dis.append(i)
        term_dis.append(term_temp)
print ("---------------------------------------------------------------------------------------------------------------------")
print ("index of updated rows:\n") 
print (ix_dis) 
print ("---------------------------------------------------------------------------------------------------------------------")
print ("updated terminals:\n") 
print (term_dis)

---------------------------------------------------------------------------------------------------------------------
index of updated rows:

[223, 354, 584, 901, 914, 939, 1598, 2668, 2928, 2667, 3254, 2925, 1002, 1001, 1533, 1701, 2665, 2664, 3253, 3240, 2958, 1515, 1623, 2660, 1622, 2811, 1517, 2722, 1693, 2920, 1000, 2919, 1531, 3252, 2195, 3250, 3251, 2197, 2917, 1894, 1895, 1514, 3107, 2194, 2916, 2659, 1683, 3273]
---------------------------------------------------------------------------------------------------------------------
updated terminals:

['Huelva', 'Taboguilla Terminal', 'Melones Terminal', 'Lianyungang', 'Petroleros Torre Arenillas Terminal', 'Aabenraa', 'Port Said', 'Palos de la Frontera Terminal', 'Bristol Aviation Fuel Terminal', 'Reina Sofia CEPSA Terminal', 'Cam Pha', 'Bristol', 'Romano Port', 'Vlore', 'Havana', 'Kiel', 'Cadiz', 'Rota', 'B12 Oil Terminal', 'Hai Phong', 'Falmouth', 'Caldera_Costa Rica', 'Kemi', 'Cape Trafalgar', 'Oulu', 'Sevketiye', 'Punta Moral

In [95]:
print ("---------------------------------------------------------------------------------------------------------------------")
print ("updated terminals rows:\n") 
df_updated=df_new.loc[ix_dis]
df_updated.tail()

---------------------------------------------------------------------------------------------------------------------
updated terminals rows:



,ix,Region,Country,Port Group,Terminal,Lat,Lon,num of call all,num of call end,num of call start,Main_ports
2194,2935,Mediterranean,Montenegro,Ploce,Bar,42.09338,19.07894,2,1,1,Romano Port
2916,3257,North sea,United Kingdom,Fawley Refinery,Bristol Cruise Terminal,51.50865,-2.70630,2,1,1,Pembroke
2659,3038,SW Europe,Spain,Cepsa Algeciras Terminal,Atlantic Copper Terminal,37.17485,-6.91807,2,1,1,Huelva
1683,2989,Baltic,Germany,Wilhelmshaven,German Naval Yards Kiel,54.32277,10.15097,2,1,1,Kalundborg
3273,2901,Mediterranean,unknown,Fertile Crescent,Fertile Crescent,32.84876,31.84852,1,1,0,Port Said


In [96]:
print (df_new[df_new.Terminal=="Bristol Cruise Terminal"])
print ("---------------------------------------------------------------------------------------------------------------------")
df_new[df_new.Terminal=="Pembroke"]

        ix     Region         Country       Port Group  \
2916  3257  North sea  United Kingdom  Fawley Refinery   

                     Terminal       Lat     Lon  num of call all  \
2916  Bristol Cruise Terminal  51.50865 -2.7063                2   

      num of call end  num of call start Main_ports  
2916                1                  1   Pembroke  
---------------------------------------------------------------------------------------------------------------------


,ix,Region,Country,Port Group,Terminal,Lat,Lon,num of call all,num of call end,num of call start,Main_ports
102,102,North sea,United Kingdom,Pembroke,Pembroke,51.69874,-5.03446,2159,1078,1081,Pembroke


### Obviously, the new updated port file is more reasonable; 1) Main port assigning is based on region categories; 2) within the same region, assigning is based on the number of calls sequence. 

### Possible further updating: 1) region is manully selected in Tableau, not so accurate; 2) there are still some overlapping terminals by using recursive. The reason is not clear.
file :   "ports_update1"

# d) rename and save data

In [97]:
# further remove some unrelecant column
df_new.drop(["ix","Port Group","num of call end","num of call start"],axis=1,inplace=True)
# rename columns
df_new.columns=["Region","Country","Terminal","Lat","Lon","num of call all","Port Group"]
df_new.to_csv("C:/posDoc-SDU-Denmark/SDU_research/shippingLabwp1/savedFile/ports_update1.csv",index=False)

In [98]:
# save to xlsx format.
csvToxlsx()
df_new.head()

,Region,Country,Terminal,Lat,Lon,num of call all,Port Group
0,SE Asia,Singapore,Singapore,1.20470,103.67232,14768,Singapore
1,Baltic,Russia,PTP Terminal,60.32058,28.72898,7566,PTP Terminal
2,Black sea,Russia,Novorossiysk,44.72680,37.78154,7524,Novorossiysk
3,NE Asia,South Korea,Yeosu,34.71473,127.81125,7078,Yeosu
4,Gulf,United States of America,Houston,29.23550,-94.62350,6746,Houston
